In [1]:
import os
from pathlib import Path
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
from skorch import NeuralNetClassifier

from dataset import *
from model import *


In [2]:
torch.__version__

'1.12.1'

In [3]:
DATA_DIR = Path("../datasets/plankton_cocov2")
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')
VALID_DIR = os.path.join(DATA_DIR, 'valid')

In [4]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

train_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.ColorJitter(p=0.2),
    A.HorizontalFlip(p=0.5),
    ToTensorV2(),
])
test_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    ToTensorV2(),
])
train_ds = PlanktonSegmentationDataset(TRAIN_DIR, transform=train_transform)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [5]:
len(train_ds)

227

In [6]:
image, mask = train_ds[1]
image.shape, mask.shape

(torch.Size([3, 224, 224]), torch.Size([1, 224, 224]))

In [7]:
unique, counts = np.unique(mask, return_counts=True)
dict(zip(unique, counts))

{0.0: 31594, 1.0: 18582}

In [9]:
net = NeuralNetClassifier(
    UNetModule,
    max_epochs=1,
    criterion=nn.BCEWithLogitsLoss,
    lr=0.1,
    iterator_train__shuffle=True,
    train_split=None,
    # device='cuda'
)
net.fit(train_ds, y=None)

KeyboardInterrupt: 

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=UNetModule(
    (model): Unet(
      (encoder): EfficientNetEncoder(
        (_conv_stem): Conv2dStaticSamePadding(
          3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
          (static_padding): ZeroPad2d((0, 1, 0, 1))
        )
        (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_blocks): ModuleList(
          (0): MBConvBlock(
            (_depthwise_conv): Conv2dStaticSamePadding(
              40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
              (static_padding): ZeroPad2d((1, 1, 1, 1))
            )
            (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
            (_se_reduce): Conv2dStaticSamePadding(
              40, 10, kernel_size=(1, 1), stride=(1, 1)
              (static_padding): Identity()
            )
            (_se_expand): Con